<br><br>

## **Import necessary Python libraries and modules**

First, we will import necessary Python libraries and modules. These include as `gdown`, for downloading large files from Google Drive (where we will get our UCSD Goodreads reviews), as well as scikit-learn (`sklearn`) and PyTorch (`torch`), for various machine learning tools.

In [1]:

# For data manipulation and analysis
import pandas as pd
import numpy as np

# For machine learning tools and evaluation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report


# For deep learning
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
import torch



To use the HuggingFace [`transformers` Python library](https://huggingface.co/transformers/installation.html), we will install it with `pip`.

In [2]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 8.1 MB/s 
     |████████████████████████████████| 596 kB 32.6 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 67 kB 2.6 MB/s 
     |████████████████████████████████| 6.8 MB 31.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Once `transformers` is installed, we will import modules for `DistilBert`, a *distilled* or smaller version of a BERT model that runs more quickly and uses less computing power. This makes it ideal for those just getting started with BERT.

In [3]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

<br><br>

## **Set parameters and file paths**

In [4]:
# This is the name of the BERT model that we want to use. 
# We're using DistilBERT to save space (it's a distilled version of the full BERT model), 
# and we're going to use the cased (vs uncased) version.
model_name = 'distilbert-base-multilingual-cased'  

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'       

# This is the maximum number of tokens in any document sent to BERT.
max_length = 512                                                        

# This is the name of the directory where we'll save our model. You can name it whatever you want.
#cached_model_directory_name = 'ABSA_FineTuning_BERT'  
cached_model_directory_name = 'Polarity_dev'  

In [5]:
#stiamo utlizzando la GPU?
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [6]:
dev=pd.read_csv("dev.csv")
dev

,Unnamed: 0,Sentence,Polarity_Classification
0,0,This thought races through Arthur's mind as he...,Neutra
1,1,Arthur is put through a rigorous test of brave...,Negativa
2,2,I recommend that anyone looking to have some m...,Neutra
3,3,"Unfortunately, I was a little surprised, and v...",Negativa
4,4,I read this book in high school and loved it,Positiva
...,...,...,...
3013,3013,3점은 됩니다....,Negativa
3014,3014,이걸 영화라고...,Negativa
3015,3015,화면이 나눠져서...보는 내내 불편했음,Negativa
3016,3016,괜히 머리 밀고 열심히 한것같다만...망작...,Negativa


In [7]:
test=pd.read_csv("test.csv")
test

,Unnamed: 0,Sentence,Polarity_Classification
0,0,Garth Nix's 'Mister Monday' was a highly cleve...,Positiva
1,1,"Instead, it appears to have been written by a ...",Negativa
2,2,"If you treated him as an equal, he could help ...",Neutra
3,3,Diabola becomes mad and uses her powers to mak...,Negativa
4,4,This is a 361 page book about a boy named Arth...,Neutra
...,...,...,...
8245,8245,이걸 영화라고..쓰레기도 이런 쓰레기는..ㅋㅋㅋ,Negativa
8246,8246,이렇게 대충 에로물 비스무리 만들어서 신인들 얼굴 도장이나 찍게 만드는 목적인 것 ...,Negativa
8247,8247,난 왜 보다가 졸앗지..,Negativa
8248,8248,이거 지대론데,Positiva


In [8]:
#creiamo le liste di train e test. 1.8k train 250 test  
train_texts=dev['Sentence']
train_labels=dev['Polarity_Classification']
test_texts=test['Sentence']
test_labels=test['Polarity_Classification']


<br><br>

## **Split the data into training and test sets**

In [9]:
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(3018, 3018, 8250, 8250)

Here's an example of a training label and review:

In [10]:
train_labels[0], train_texts[0]

('Neutra',
 "This thought races through Arthur's mind as he saves the world in Mister Monday")

<br><br>

## **Encode data for BERT**

We're going to transform our texts and labels into a format that BERT (via Huggingface and PyTorch) will understand. This is called *encoding* the data.

Here are the steps we need to follow:

1. The labels&mdash;in this case, Goodreads genres&mdash;need to be turned into integers rather than strings.

2. The texts&mdash;in this case, Goodreads reviews&mdash;need to be truncated if they're more than 512 tokens or padded if they're fewer than 512 tokens. The tokens, or words in the texts, also need to be separated into "word pieces" and matched to their embedding vectors.

3. We need to add special tokens to help BERT:

| BERT special token | Explanation |
| --------------| ---------|
| [CLS] | Start token of every document. |
| [SEP] | Separator between each sentence |
| [PAD] | Padding at the end of the document as many times as necessary, up to 512 tokens |
|  &#35;&#35; | Start of a "word piece" |




Here we will load `DistilBertTokenizerFast` from the HuggingFace library, which will do all the work of encoding the texts for us. The `tokenizer()` will break word tokens into word pieces, truncate to 512 tokens, and add padding and special BERT tokens.

In [11]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Here we will create a map of our labels, or Goodreads genres, to integer keys. We take the unique labels, and then we make a dictionary that associates each label/tag with an integer.

**Note:** HuggingFace documentation sometimes refers to "labels" as "tags" but these are the same thing. We use "labels" throughout this notebook for clarity.

In [12]:
unique_labels={'Neutra', 'Positiva', 'Negativa'}
label2id={'Negativa': 2, 'Neutra': 0, 'Positiva': 1}
id2label={2:'Negativa', 0:'Neutra', 1:'Positiva'}

In [13]:
label2id.keys()

dict_keys(['Negativa', 'Neutra', 'Positiva'])

In [14]:
id2label.keys()

dict_keys([2, 0, 1])

Now let's encode our texts and labels!

In [15]:
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
test_encodings  = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=max_length)

train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]

In [16]:
type(test_labels_encoded[100])

int

**Examine a Goodreads review in the training set after encoding**

In [21]:
' '.join(train_encodings[29].tokens[0:100])

'[CLS] In this fant ##astic story of trust and adventure ##s , a boy named Author holds the Key to the future , and what will that bring ? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

**Examine a Goodreads review in the test set after encoding**

In [22]:
' '.join(test_encodings[0].tokens[0:100])

"[CLS] Ga ##rth Ni ##x ' s ' Mister Monday ' was a highly c ##lever , creative , and enter ##taining read that had me up into the we ##e hours of the morning following Arthur ' s adventure ##s in the House [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

**Examine the training labels after encoding**

In [17]:
set(train_labels_encoded)

{0, 1, 2}

**Examine the test labels after encoding**

In [18]:
set(test_labels_encoded)

{0, 1, 2}

<br><br>

## **Make a custom Torch dataset**

Here we combine the encoded labels and texts into dataset objects. We use the custom Torch `MyDataSet` class to make a `train_dataset` object from  the `train_encodings` and `train_labels_encoded`. We also make a `test_dataset` object from `test_encodings`, and `test_labels_encoded`.

In [18]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [19]:
train_dataset = MyDataset(train_encodings, train_labels_encoded)
test_dataset = MyDataset(test_encodings, test_labels_encoded)

In [27]:
' '.join(train_dataset.encodings[0].tokens[0:100])

"[CLS] This thought races through Arthur ' s mind as he save ##s the world in Mister Monday [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

In [28]:
' '.join(test_dataset.encodings[1].tokens[0:100])

"[CLS] Instead , it appears to have been written by a woman with a chip on her shoulder with a pes ##sim ##istic view of society and women ' s abilities to think for themselves [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

<br><br>

## **Load pre-trained BERT model**

Here we load a pre-trained DistilBERT model and send it to CUDA.

**Note:** If you decide to repeat fine-tuning after already running the following cells, make sure that you re-run this cell to re-load the original pre-trained model before fine-tuning again.

In [20]:
# The model_name needs to match the name used for the tokenizer above.
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'pre_classifie

<br><br>

## **Set the BERT fine-tuning parameters**

These are the arguments we'll set in the HuggingFace TrainingArguments objects, which we'll then pass to the HuggingFace Trainer object. There are many more possible arguments, but here we highlight the basics and some common gotchas.

When training your own model, you should search over these parameters to find the best settings for your particular dataset. You should use a held-out set of validation data for this step.

| Parameter | Explanation |
|-----------| ------------|
| num_train_epochs | total number of training epochs (how many times to pass through the entire dataset; too much can cause overfitting) |
| per_device_train_batch_size | batch size per device during training |
| per_device_eval_batch_size |  batch size for evaluation |
|  warmup_steps |  number of warmup steps for learning rate scheduler (set lower because of small dataset size) |
| weight_decay | strength of weight decay (reduces size of weights, like regularization) |
| output_dir | output directory for the fine-tuned model and configuration files |
| logging_dir | directory for storing logs |
| logging_steps | how often to print logging output (so that we can stop training early if the loss isn't going down) |
| evaluation_strategy | evaluate while training so that we can see the accuracy going up |

In [21]:
training_args = [TrainingArguments(
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps'     # evaluate during fine-tuning so that we can see progress
), TrainingArguments(
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps'     # evaluate during fine-tuning so that we can see progress
), 
TrainingArguments(
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    learning_rate=3e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps'     # evaluate during fine-tuning so that we can see progress
),
TrainingArguments(
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps'     # evaluate during fine-tuning so that we can see progress
),
TrainingArguments(
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps'     # evaluate during fine-tuning so that we can see progress
),
TrainingArguments(
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=3e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps'     # evaluate during fine-tuning so that we can see progress
)]

<br><br>

## **Fine-tune the BERT model**

First, we define a custom evaluation function that returns the accuracy. You could modify this function to return precision, recall, F1, and/or other metrics.

In [22]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

Then we create a HuggingFace `Trainer` object using the `TrainingArguments` object that we created above. We also send our `compute_metrics` function to the `Trainer` object, along with our test and train datasets.

**Note:** This is what we've been aiming for this whole time! All the work of tokenizing, creating datasets, and setting the training arguments was for this cell.

In [23]:
for argomenti_train in training_args:
  trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=argomenti_train,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset (usually a validation set; here we just send our test set)
    compute_metrics=compute_metrics      # our custom evaluation function 
    )
  
  trainer.train()

  trainer.evaluate()

  predicted_results = trainer.predict(test_dataset)

  predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
  predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
  predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

  print(classification_report(test_labels, 
                            predicted_labels))
  print('\n')


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3018
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 190


Step,Training Loss,Validation Loss,Accuracy
100,0.791000,0.630468,0.691758


***** Running Evaluation *****
  Num examples = 8250
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 8250
  Batch size = 32


***** Running Prediction *****
  Num examples = 8250
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3018
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 190


              precision    recall  f1-score   support

    Negativa       0.71      0.78      0.74      3617
      Neutra       0.64      0.41      0.50       469
    Positiva       0.78      0.74      0.76      4164

    accuracy                           0.74      8250
   macro avg       0.71      0.65      0.67      8250
weighted avg       0.74      0.74      0.74      8250





...

              precision    recall  f1-score   support

    Negativa       0.75      0.77      0.76      3617
      Neutra       0.59      0.53      0.56       469
    Positiva       0.79      0.78      0.79      4164

    accuracy                           0.76      8250
   macro avg       0.71      0.69      0.70      8250
weighted avg       0.76      0.76      0.76      8250





Step,Training Loss,Validation Loss,Accuracy
100,0.022300,1.752461,0.748485
200,0.038800,1.831410,0.751636
300,0.029700,1.894681,0.751152


***** Running Evaluation *****
  Num examples = 8250
  Batch size = 16
***** Running Evaluation *****
  Num examples = 8250
  Batch size = 16
***** Running Evaluation *****
  Num examples = 8250
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 8250
  Batch size = 16


***** Running Prediction *****
  Num examples = 8250
  Batch size = 16


              precision    recall  f1-score   support

    Negativa       0.74      0.77      0.76      3617
      Neutra       0.56      0.55      0.55       469
    Positiva       0.79      0.77      0.78      4164

    accuracy                           0.76      8250
   macro avg       0.70      0.70      0.70      8250
weighted avg       0.76      0.76      0.76      8250





<br><br>

## **Save fine-tuned model**

The following cell will save the model and its configuration files to a directory in Colab. To preserve this model for future use, you should download the model to your computer.

In [ ]:
trainer.save_model(cached_model_directory_name)

Saving model checkpoint to Polarity_1
Configuration saved in Polarity_1/config.json
Model weights saved in Polarity_1/pytorch_model.bin


(Optional) If you've already fine-tuned and saved the model, you can reload it using the following line. You don't have to run fine-tuning every time you want to evaluate.

<br><br>

## **Evaluate fine-tuned model**

The following function of the `Trainer` object will run the built-in evaluation, including our `compute_metrics` function.

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 63404
  Batch size = 32


{'epoch': 4.0,
 'eval_accuracy': 0.1874171976531449,
 'eval_loss': 1.3463501930236816,
 'eval_runtime': 1137.9764,
 'eval_samples_per_second': 55.716,
 'eval_steps_per_second': 1.742}

But we might want to do more fine-grained analysis of the model, so we extract the predicted labels.

In [ ]:
predicted_results = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 63404
  Batch size = 32


In [ ]:
predicted_results.predictions.shape

(63404, 3)

In [ ]:
predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

In [ ]:
len(predicted_labels)

63404

In [ ]:
print(classification_report(test_labels, 
                            predicted_labels))

              precision    recall  f1-score   support

    Negativa       0.64      0.02      0.03     31245
      Neutra       0.01      0.80      0.02       453
    Positiva       0.71      0.35      0.47     31706

    accuracy                           0.19     63404
   macro avg       0.45      0.39      0.17     63404
weighted avg       0.67      0.19      0.25     63404



In [ ]:
#valutazione per domain

In [ ]:
def domain_evaluation(test_subset):
  predicted_results = trainer.predict(test_subset)
  predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
  predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
  predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability
  return classification_report(test_labels, predicted_labels)

In [ ]:
test_dataset 

In [ ]:
#creiamo i subset del train
en_test=test_dataset[:616]
it_test=test_dataset[616:3637]
ko_test=test_dataset[3637:]

In [ ]:
domain_evaluation(en_test)

***** Running Prediction *****
  Num examples = 3
  Batch size = 32


KeyError: ignored

In [ ]:
domain_evaluation(it_test)

In [ ]:
domain_evaluation(ko_test)

In [ ]:
#RELOAD THE MODEL

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
#tok = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.
tok = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.
mod = DistilBertForSequenceClassification.from_pretrained(cached_model_directory_name)

loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/28e5b750bf4f39cc620367720e105de1501cf36ec4ca7029eba82c1d2cc47caf.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/5cbdf121f196be5f1016cb102b197b0c34009e1e658f513515f2eebef9f38093.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/47087d99feeb3bc6184d7576ff089c52f7fbe

In [ ]:
#come prova gli passo delle recensioni
revita=pd.read_csv('anobii_rev_ita.csv')
revita

,Unnamed: 0,index,Unnamed: 0.1,Genere,Title,ISBN,Username,Rating,Date,N_Likes,Review_Content,Language_AN
0,0,0,0,Children_Lit,11 favole di felicità. Imparare a pensare posi...,978-8859003069,I.C.Manzoni-Radice Lucera,error,"Nov 3, 2014,",0,Attraverso 11 favole buffe e numerosi personag...,it
1,1,1,1,Children_Lit,A caccia dell'Orso. Ediz. a colori,978-8804702535,marin,error,"Dec 27, 2010,",1,"E' una storia avvincente, scritta come fosse u...",it
2,2,2,2,Children_Lit,A caccia dell'Orso. Ediz. a colori,978-8804702535,Sbrinedda,error,"Mar 26, 2012, 1",1,Peccato che la versione in italiano sia introv...,it
3,3,3,3,Children_Lit,A caccia dell'Orso. Ediz. a colori,978-8804702535,Max e Ruby,error,"Oct 25, 2013,",0,Bellissimo! Non può mancare nella libreria di ...,it
4,4,4,4,Children_Lit,A caccia dell'Orso. Ediz. a colori,978-8804702535,Scaffale Basso,5,"Jan 29, 2014, 1",0,Il ritmo dell’avventura è scandito da un testo...,it
...,...,...,...,...,...,...,...,...,...,...,...,...
63752,63752,63796,63796,Thriller,Vipera. Nessuna resurrezione per il commissari...,978-8806220969,alduccio,error,"Dec 16, 2012, 1",2,L'incipit numero uno: la domanda dell'assassin...,it
63753,63753,63797,63797,Thriller,Vipera. Nessuna resurrezione per il commissari...,978-8806220969,Annalisa (su anobii che peggiora ogni volta ch...,error,"Nov 30, 2012,",5,E se da una parte ritroviamo tutti i ‘nostri’ ...,it
63754,63754,63798,63798,Thriller,Vipera. Nessuna resurrezione per il commissari...,978-8806220969,Francesca P. (Cassie),error,"Nov 30, 2012,",14,De Giovanni ha detto più di una volta che scri...,it
63755,63755,63799,63799,Thriller,Vipera. Nessuna resurrezione per il commissari...,978-8806220969,"Anina e ""gambette di pollo""",error,"Nov 28, 2012,",17,"Tra supporters ed anteprime, de Giovanni sta d...",it


In [ ]:
out_list=[]
sen_list=[]
rev_list=revita[:5]['Review_Content']
for rev in rev_list:
    sentences=rev.split(".") #spezziamo la recensione in frasi
    lun=len(sentences) #ci salviamo la lunghezza della lista
    #tokenizziamo
    for sen in sentences:
      sen_list.append(sen)
      encoded_input = tok(sen, return_tensors='pt',truncation=True, padding='max_length')
      output = mod(**encoded_input)
      out_list.append(output)

pd.DataFrame(list(zip(sen_list, out_list)))
#print(len(out_list))

#0=neutra, 1=positiva, 2=negativa

,0,1
0,Attraverso 11 favole buffe e numerosi personag...,"{'logits': [[tensor(-0.1684, grad_fn=<UnbindBa..."
1,Un porcospino irascibile che non riesce a trat...,"{'logits': [[tensor(-0.2170, grad_fn=<UnbindBa..."
2,Alla fine di ogni favola sono proposte inoltre...,"{'logits': [[tensor(0.5204, grad_fn=<UnbindBac..."
3,Queste storie traggono ispirazione dalla nota ...,"{'logits': [[tensor(0.3810, grad_fn=<UnbindBac..."
4,,"{'logits': [[tensor(0.2103, grad_fn=<UnbindBac..."
5,"E' una storia avvincente, scritta come fosse u...","{'logits': [[tensor(-0.3282, grad_fn=<UnbindBa..."
6,E' divertente e fà ridere per i suoi suoni bu...,"{'logits': [[tensor(-0.9151, grad_fn=<UnbindBa..."
7,In rete c'è il video del libro recitato dall'...,"{'logits': [[tensor(0.6955, grad_fn=<UnbindBac..."
8,Peccato che la versione in italiano sia introv...,"{'logits': [[tensor(-1.3555, grad_fn=<UnbindBa..."
9,Bellissimo! Non può mancare nella libreria di ...,"{'logits': [[tensor(-1.3869, grad_fn=<UnbindBa..."


In [ ]:
out_list  #nel tensor abbiamo le 'probabilità' che la sentece venga classificata come neutra positiva o negativa. La piu alta vince

[SequenceClassifierOutput([('logits',
                            tensor([[-0.1684,  0.0517,  0.4594]], grad_fn=<AddmmBackward0>))]),
 SequenceClassifierOutput([('logits',
                            tensor([[-0.2170, -0.4670,  1.1866]], grad_fn=<AddmmBackward0>))]),
 SequenceClassifierOutput([('logits',
                            tensor([[ 0.5204, -0.1430,  0.1376]], grad_fn=<AddmmBackward0>))]),
 SequenceClassifierOutput([('logits',
                            tensor([[ 0.3810, -0.1144,  0.1192]], grad_fn=<AddmmBackward0>))]),
 SequenceClassifierOutput([('logits',
                            tensor([[ 0.2103,  0.8336, -0.4596]], grad_fn=<AddmmBackward0>))]),
 SequenceClassifierOutput([('logits',
                            tensor([[-0.3282,  0.3390,  0.2520]], grad_fn=<AddmmBackward0>))]),
 SequenceClassifierOutput([('logits',
                            tensor([[-0.9151,  1.3228,  0.0127]], grad_fn=<AddmmBackward0>))]),
 SequenceClassifierOutput([('logits',
                        

In [ ]:
sen_list

['Attraverso 11 favole buffe e numerosi personaggi divertenti in cui ci si potrà facilmente identificare, questo libro insegna ai bambini, ma non solo a loro, come riconoscere e modificare quei comportamenti e quei pensieri negativi che ostacolano il benessere personale',
 'Un porcospino irascibile che non riesce a trattenersi dal lanciare le proprie spine, una giraffa sognatrice ma piuttosto inconcludente, un camaleonte insicuro che cambia colore in continuazione pur di farsi accettare dagli altri, un ranocchio che non si piace affatto e che desidera a tutti i costi diventare principe, un coccodrillo piagnucolone che rimugina sul suo passato e immagina catastrofi future… sono soltanto alcune delle tante bizzarre bestioline protagoniste di questi racconti, tutti ideati per strappare un sorriso, per promuovere il piacere di leggere e per far arrivare il messaggio che la felicità è possibile, a condizione di impegnarsi per modificare le cattive abitudini',
 'Alla fine di ogni favola sono

In [ ]:
#bisogna però valutare il punteggio finale per la review, e quindi trovare un modo per determinare questo punteggio
#valutare la migliore maniera per splittare le recensioni

In [ ]:
#ESPERIMENTO ZERO SHOT CROSS-LINGUAL TRANSFER LEARNING

In [ ]:
#pipeline:

# traduco le sentences del dataset di training in italiano ed in coreano con il deep translator
#le passo al modello trainato 
# valuto l'output

import time

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
#tok = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.
tok = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.
mod = DistilBertForSequenceClassification.from_pretrained(cached_model_directory_name)

In [ ]:
#importo il dataset FT_ABSA
import pandas as pd
trainset=pd.read_csv("C:\\Users\\Fossati\\Desktop\\Tesi\\Dati\\Dati_FineTuningBERT\\FT_ABSA.csv")
trainset

,Unnamed: 0,index,Sentence,Category,Polarity_Category,Polarity_Score,Polarity_Classification
0,0,0,The solution given here is fantastic and a bit...,['BOOK#GENERAL'],['positive'],1.000,Positiva
1,1,1,This thought races through Arthur's mind as he...,"['CONTENT#CHARACTERS', 'BOOK#TITLE']","['neutral', 'neutral']",0.000,Neutra
2,2,2,Arthur is put through a rigorous test of brave...,"['CONTENT#CHARACTERS', 'CONTENT#CHARACTERS']","['neutral', 'negative']",-0.500,Negativa
3,3,3,"In the story, Arthur receives a key that gives...","['CONTENT#CHARACTERS', 'CONTENT#PLOT', 'CONTEN...","['neutral', 'neutral', 'neutral', 'neutral', '...",0.000,Neutra
4,4,4,Arthur has to go to a foreign realm and defeat...,"['CONTENT#CHARACTERS', 'CONTENT#PLOT', 'CONTEN...","['neutral', 'neutral', 'neutral', 'neutral']",0.000,Neutra
...,...,...,...,...,...,...,...
2046,2046,2321,"Arthur, and young boy that was not athletic an...","['CONTENT#CHARACTERS', 'CONTENT#PLOT', 'CONTEN...","['positive', 'neutral', 'neutral']",0.333,Positiva
2047,2047,2322,I recommend it to any other readers,['BOOK#GENERAL'],['positive'],1.000,Positiva
2048,2048,2323,Best of the Best,['BOOK#GENERAL'],['positive'],1.000,Positiva
2049,2049,2324,"From the award-winning author of ""Snow Falling...","['BOOK#AUTHOR', 'CONTENT#PLOT', 'CONTENT#PLOT']","['positive', 'neutral', 'neutral']",0.333,Positiva


In [ ]:
eng_set=trainset[['Sentence','Polarity_Classification']]
eng_set

,Sentence,Polarity_Classification
0,The solution given here is fantastic and a bit...,Positiva
1,This thought races through Arthur's mind as he...,Neutra
2,Arthur is put through a rigorous test of brave...,Negativa
3,"In the story, Arthur receives a key that gives...",Neutra
4,Arthur has to go to a foreign realm and defeat...,Neutra
...,...,...
2046,"Arthur, and young boy that was not athletic an...",Positiva
2047,I recommend it to any other readers,Positiva
2048,Best of the Best,Positiva
2049,"From the award-winning author of ""Snow Falling...",Positiva


In [ ]:
#creazione lista italiana

from deep_translator import GoogleTranslator
it_rev_list=[]
for el in eng_set['Sentence'][:500].tolist():
    to_translate = el
    #print(rev_list[1])
    #print("\n")
    translated = GoogleTranslator(source='en', target='it').translate(to_translate)
    #print(translated)
    it_rev_list.append(translated)
print(len(it_rev_list))

500


In [ ]:
it_rev_list[:10]

["La soluzione data qui è fantastica e un po' bizzarra e qui sta il fascino di questo libro",
 'Questo pensiero corre nella mente di Arthur mentre salva il mondo in Mister Monday',
 'Arthur è sottoposto a una rigorosa prova di coraggio e compassione per affrontare il malvagio Mister Monday',
 "Nella storia, Arthur riceve una chiave che gli dà il potere di cui ha bisogno per sconfiggere Mister Monday, l'unico problema è che non sa chi sia Monday, o niente dello strano mondo in cui vive Monday",
 "Arthur deve andare in un regno straniero e sconfiggere un nemico che ha molto più potere di lui, se Monday fosse un T-Rex allora Arthur sarebbe un filo d'erba rispetto a lui",
 'La battaglia di Arthur per salvare la terra richiede un livello di coraggio che immagina solo nei suoi sogni',
 'Ecco perché il tema è il coraggio, Arthur deve evocare un livello di coraggio che non può e, attraverso la storia, lotta per ottenere questo coraggio sconosciuto',
 'Arthur si avventura nella storia trovando 

In [ ]:
#creazione lista koreana

from deep_translator import GoogleTranslator
ko_rev_list=[]
for el in eng_set['Sentence'][:500].tolist():
    to_translate = el
    #print(rev_list[1])
    #print("\n")
    translated = GoogleTranslator(source='en', target='ko').translate(to_translate)
    #print(translated)
    ko_rev_list.append(translated)
print(len(ko_rev_list))

500


In [ ]:
ko_rev_list[:10]

['여기에 주어진 해결책은 환상적이고 약간 기괴하며 이 책의 매력이 있습니다.',
 '이 생각은 미스터 먼데이에서 세상을 구하는 Arthur의 마음을 뛰어넘습니다.',
 'Arthur는 사악한 Mister Monday에 맞서기 위해 엄격한 용기와 연민의 시험을 받습니다.',
 '이야기에서 아서는 미스터 먼데이를 물리치는 데 필요한 힘을 주는 열쇠를 받습니다. 유일한 문제는 월요일이 누구인지 또는 월요일이 사는 이상한 세계에 대해 아무것도 모른다는 것입니다.',
 'Arthur는 외국에 가서 자신보다 훨씬 더 많은 힘을 가진 적을 물리쳐야 합니다. 월요일이 T-Rex라면 Arthur는 그에게 비하면 풀잎일 것입니다.',
 '지구를 구하기 위한 Arthur의 전투에는 꿈에서만 상상할 수 있는 수준의 용기가 필요합니다.',
 '그래서 테마가 용기인데, 아서는 자신이 할 수 없는 수준의 용기를 불러내야 하고, 스토리를 통해 이 미지의 용기를 얻기 위해 고군분투한다.',
 'Arthur는 자신에게 필요한 용기를 찾기 위해 이야기를 모험합니다.',
 'Mister Monday는 우수한 10/10을 얻은 경이로운 책입니다.',
 '이 책은 이야기를 마무리합니다. 그것은 블랙홀처럼 당신을 페이지 속으로 빨아들입니다']

In [ ]:
#l'esperimento prevede la divisione del set di recensioni in 5 gruppi da 100 recesnioni. Si fa agire il modello e si vedono i risultati

In [ ]:
gruppi=[it_rev_list[:100],it_rev_list[100:200],it_rev_list[200:300],it_rev_list[300:400],it_rev_list[400:500]]
count=0
results=[]
for gruppo in gruppi:
    count+=1 #per tenere conto del gruppo in cui siamo
    it_out_list=[]  #rinnoviamo la lista
    
    for rev in gruppo:
        encoded_input = tok(rev, return_tensors='pt',truncation=True, padding='max_length')
        output = mod(**encoded_input)
        it_out_list.append(output)
        time.sleep(1)


    predictions=[]
    for el in it_out_list:
            pred=el[0][0].argmax()
            if pred == 1:
                predictions.append('Positiva')
            elif pred == 0:
                predictions.append('Neutra')
            else:
                predictions.append('Negativa')
                
    if count==1:
            en_it=pd.DataFrame(list(zip(predictions,eng_set['Polarity_Classification'][:100])), columns=['en_pol','it_pol'])
            equal=0
            for index,row in en_it.iterrows():
                if row['en_pol'] == row['it_pol']:
                    equal+=1
                    
            results.append(equal)

    elif count==2:
            en_it=pd.DataFrame(list(zip(predictions,eng_set['Polarity_Classification'][100:200])), columns=['en_pol','it_pol'])
            equal=0
            for index,row in en_it.iterrows():
                if row['en_pol'] == row['it_pol']:
                    equal+=1
                    
            results.append(equal)
            
    elif count==3:
            en_it=pd.DataFrame(list(zip(predictions,eng_set['Polarity_Classification'][200:300])), columns=['en_pol','it_pol'])
            equal=0
            for index,row in en_it.iterrows():
                if row['en_pol'] == row['it_pol']:
                    equal+=1
                    
            results.append(equal)
            
    elif count==4:
            en_it=pd.DataFrame(list(zip(predictions,eng_set['Polarity_Classification'][300:400])), columns=['en_pol','it_pol'])
            equal=0
            for index,row in en_it.iterrows():
                if row['en_pol'] == row['it_pol']:
                    equal+=1
                    
            results.append(equal)
            
    elif count==5:
            en_it=pd.DataFrame(list(zip(predictions,eng_set['Polarity_Classification'][400:500])), columns=['en_pol','it_pol'])
            equal=0
            for index,row in en_it.iterrows():
                if row['en_pol'] == row['it_pol']:
                    equal+=1
                    
            results.append(equal)
            

            
import numpy as np
print(results)
print(np.mean(results))



  






#0=neutra, 1=positiva, 2=negativa

[74, 68, 76, 76, 67]
72.2


In [ ]:
it_out_list[2][0][0].argmax()

tensor(1)

In [ ]:
#72 su 100 sono corrette in media dall'esperimento in media

In [ ]:
#l'esperimento per il coreano

In [ ]:
gruppi=[ko_rev_list[:100],ko_rev_list[100:200],ko_rev_list[200:300],ko_rev_list[300:400],ko_rev_list[400:500]]
count=0
results=[]
for gruppo in gruppi:
    count+=1 #per tenere conto del gruppo in cui siamo
    ko_out_list=[]  #rinnoviamo la lista
    
    for rev in gruppo:
        encoded_input = tok(rev, return_tensors='pt',truncation=True, padding='max_length')
        output = mod(**encoded_input)
        ko_out_list.append(output)
        time.sleep(2)


    predictions=[]
    for el in ko_out_list:
            pred=el[0][0].argmax()
            if pred == 1:
                predictions.append('Positiva')
            elif pred == 0:
                predictions.append('Neutra')
            else:
                predictions.append('Negativa')
                
    if count==1:
            en_it=pd.DataFrame(list(zip(predictions,eng_set['Polarity_Classification'][:100])), columns=['en_pol','ko_pol'])
            equal=0
            for index,row in en_it.iterrows():
                if row['en_pol'] == row['ko_pol']:
                    equal+=1
                    
            results.append(equal)
            print(equal)

    elif count==2:
            en_it=pd.DataFrame(list(zip(predictions,eng_set['Polarity_Classification'][100:200])), columns=['en_pol','ko_pol'])
            equal=0
            for index,row in en_it.iterrows():
                if row['en_pol'] == row['ko_pol']:
                    equal+=1
                    
            results.append(equal)
            print(equal)
            
    elif count==3:
            en_it=pd.DataFrame(list(zip(predictions,eng_set['Polarity_Classification'][200:300])), columns=['en_pol','ko_pol'])
            equal=0
            for index,row in en_it.iterrows():
                if row['en_pol'] == row['ko_pol']:
                    equal+=1
                    
            results.append(equal)
            print(equal)
            
    elif count==4:
            en_it=pd.DataFrame(list(zip(predictions,eng_set['Polarity_Classification'][300:400])), columns=['en_pol','it_pol'])
            equal=0
            for index,row in en_it.iterrows():
                if row['en_pol'] == row['it_pol']:
                    equal+=1
                    
            results.append(equal)
            print(equal)
            
    elif count==5:
            en_it=pd.DataFrame(list(zip(predictions,eng_set['Polarity_Classification'][400:500])), columns=['en_pol','ko_pol'])
            equal=0
            for index,row in en_it.iterrows():
                if row['en_pol'] == row['ko_pol']:
                    equal+=1
                    
            results.append(equal)
            print(equal)
            

            
import numpy as np
print(results)
print(np.mean(results))



  






#0=neutra, 1=positiva, 2=negativa

64
58
64
51
56
[64, 58, 64, 51, 56]
58.6


In [ ]:
#59/100 in media. Si tenga però conto che la traduzione in koreano non è esattamente affidabile, o meglio non come quella IT